In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import xml.etree.ElementTree as ET
import pickle
import math
import copy
from rdp import *
token = "pk.eyJ1IjoibG1hZ25hbmEiLCJhIjoiY2s2N3hmNzgwMGNnODNqcGJ1N2l2ZXZpdiJ9.-aOxDLM8KbEQnJfXegtl7A"
px.set_mapbox_access_token(token)

In [3]:
def load_gpx():
    with open('gpx.df','rb') as infile:
        df = pickle.load(infile)
    begin = int(df.iloc[-1]["route_num"])
    print(begin)
    for i in range(begin+1, begin+1109+1):
        tree = ET.parse('Datas/GPS/GPX/data'+str(i)+'.gpx')
        if(len(tree.getroot()) > 1):
            root = tree.getroot()[1][0]
            df_temp = pd.DataFrame(columns=['lat', 'lon'])
            j=0
            for child in root:
                coord = child.attrib
                coord['lat'] = float(coord['lat'])
                coord['lon'] = float(coord['lon'])
                df_temp = df_temp.append(pd.DataFrame(coord, index=[j]))
                j+=1
            df_temp["route_num"] = i
            df = df.append(df_temp)
    with open('gpx.df', 'wb') as outfile:
        pickle.dump(df, outfile)

In [3]:
def rd_compression(df):
    df_simplified = pd.DataFrame(columns=['lat', 'lon', 'route_num'])
    for i in range(1, 1110):
        route = df[df['route_num']==i].values
        if(len(route)>0):
            simplified = rdp(np.delete(route, 2, 1), epsilon=1e-4)
            simplified = np.insert(simplified, 2, route[0][2], axis=1)
            df_temp = pd.DataFrame(simplified, columns=['lat', 'lon', 'route_num'])
            df_simplified = df_simplified.append(df_temp)
    return df_simplified

In [75]:
df_simplified = rd_compression(df)
with open('gpx_simplified.df', 'wb') as outfile:
    pickle.dump(df_simplified, outfile)

In [2]:
def display(file, n=75):    
    with open(file,'rb') as infile:
        df = pickle.load(infile)
    fig = px.line_mapbox(df[df["route_num"]<n], lat="lat", lon="lon", line_group="route_num", zoom=11)
    fig.show()

In [5]:
from sklearn.cluster import DBSCAN
def dbscan_clustering(df):
    X = df[["lon", "lat"]].values
    clustering = DBSCAN(eps=0.1,min_samples=2).fit(X)
    return clustering.labels_
    

In [10]:
with open("gpx.df",'rb') as infile:
    df = pickle.load(infile)
clusters = dbscan_clustering(df[df["route_num"]<5])

In [5]:
display("gpx_simplified.df", 3)